In [ ]:
!pip install duckdb

In [ ]:
%matplotlib inline

# adding project dirs to path so code may be referenced from the notebook
import sys
sys.path.insert(0, '..')

In [ ]:
import duckdb
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import config
import utils

In [ ]:
# load usgs gages
%time
gdf = utils.get_usgs_gages()

In [ ]:
gdf

In [ ]:
gdf.plot(markersize=1.5, figsize=(20,10))

In [ ]:
# Query some forcast data from parquet files
import importlib
import queries
importlib.reload(queries)

In [ ]:
query = queries.calculate_nwm_feature_metrics(
    config.MEDIUM_RANGE_PARQUET,
    config.USGS_PARQUET,
    group_by=["nwm_feature_id"],
    order_by=["observed_average"],
    filters=[
        {
            "column": "reference_time",
            "operator": "=",
            "value": "2023-01-03 12:00:00"
        },
    ]
)
print(query)
df = duckdb.query(query).to_df()
df

In [ ]:
# Join query to basins
gdf_map = gdf.merge(df, left_on="nwm_feature_id", right_on="nwm_feature_id")

# Filter to CA
# gdf_map = gdf_map.loc[gdf_map["catchment_id"].str.startswith("18")]

gdf_map.plot("observed_average", legend=True, markersize=1.5, figsize=(20,10))

In [ ]:
query = queries.get_joined_nwm_feature_timeseries(
    config.MEDIUM_RANGE_PARQUET,
    config.USGS_PARQUET,
    filters=[
        {
            "column": "reference_time",
            "operator": "=",
            "value": "2023-01-03 12:00:00"
        },
        {
            "column": "nwm_feature_id",
            "operator": "=",
            "value": "19266232"
        },
    ]
)

df = duckdb.query(query).to_df()
df

In [ ]:
ax = plt.gca()
df.plot(x= 'value_time', y="forecast_value", ax=ax, figsize=(20,10), color="blue", stacked=True)
df.plot(x= 'value_time', y="observed_value", ax=ax, figsize=(20,10), color="orange", stacked=True)